In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Загружаем модель и процессор CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Загружаем изображение
image = Image.open("path/to/your/image.jpg")

# Ввод текста
text = ["Описание 1", "Описание 2", "Описание 3"]

# Преобразуем данные
inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

# Получаем предсказания
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Результат для изображений
    probs = logits_per_image.softmax(dim=1)  # Вероятности для каждого текста

print(probs)

In [ ]:
pip install transformers torch pandas pillow

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
from tqdm import tqdm

# Загружаем модель и процессор CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Загружаем датасет
# Ожидается, что в CSV есть колонки "image_path" и "text" (путь к изображению и текстовое описание)
dataset = pd.read_csv("path/to/your/dataset.csv")

# Список для хранения результатов
results = []

# Обработка датасета
for idx, row in tqdm(dataset.iterrows(), total=len(dataset)):
    # Загружаем изображение
    image_path = row['image_path']
    text = [row['text']]
    
    try:
        image = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Ошибка при загрузке изображения {image_path}: {e}")
        continue

    # Преобразуем данные для модели
    inputs = processor(text=text, images=image, return_tensors="pt", padding=True)

    # Получаем предсказания
    with torch.no_grad():
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image  # Результаты для изображения
        probs = logits_per_image.softmax(dim=1)  # Вероятности соответствия тексту

    # Сохраняем результаты для текущей строки
    results.append({
        "image_path": image_path,
        "text": text[0],
        "probability": probs[0].item()
    })

# Конвертируем результаты в DataFrame и сохраняем в CSV
results_df = pd.DataFrame(results)
results_df.to_csv("clip_inference_results.csv", index=False)

print("Инференс завершен. Результаты сохранены в clip_inference_results.csv")


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from tqdm import tqdm

# Загружаем предварительно обученную модель и процессор
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Переключаем модель в режим обучения
model.train()

# Проверка на наличие GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:
class CustomClipDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image_path']
        text = self.data.iloc[idx]['text']
        
        # Загружаем изображение
        image = Image.open(image_path).convert("RGB")
        
        return image, text


In [ ]:
# Параметры обучения
batch_size = 8
num_epochs = 3
learning_rate = 1e-5

# Загрузка данных
dataset = CustomClipDataset("path/to/your/dataset.csv")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Оптимизатор
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Цикл обучения
for epoch in range(num_epochs):
    total_loss = 0
    for images, texts in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        
        # Преобразуем данные и отправляем на устройство
        inputs = processor(text=texts, images=images, return_tensors="pt", padding=True).to(device)
        
        # Прямой проход
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text
        
        # Целевые метки для обучения (индексная матрица, где каждый элемент соответствует своему изображению/тексту)
        labels = torch.arange(len(images)).to(device)

        # Вычисляем потери
        loss_img = torch.nn.functional.cross_entropy(logits_per_image, labels)
        loss_txt = torch.nn.functional.cross_entropy(logits_per_text, labels)
        loss = (loss_img + loss_txt) / 2

        # Обратное распространение ошибки
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

# Сохранение модели
model.save_pretrained("clip_finetuned_model")
print("Модель дообучена и сохранена в clip_finetuned_model")


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class PairDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text_a = self.data.iloc[idx]["text_a"]
        text_b = self.data.iloc[idx]["text_b"]
        label = self.data.iloc[idx]["label"]

        # Токенизируем оба текста
        encoding_a = self.tokenizer(
            text_a,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        encoding_b = self.tokenizer(
            text_b,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids_a": encoding_a["input_ids"].squeeze(),
            "attention_mask_a": encoding_a["attention_mask"].squeeze(),
            "input_ids_b": encoding_b["input_ids"].squeeze(),
            "attention_mask_b": encoding_b["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.float)
        }


In [ ]:
from transformers import AutoModel

# Загружаем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")


In [ ]:
import torch
import torch.nn.functional as F

def contrastive_loss(embeddings_a, embeddings_b, labels, margin=1.0):
    cos_sim = F.cosine_similarity(embeddings_a, embeddings_b)
    # Контрастивная потеря с целевой меткой
    loss = torch.mean((1 - labels) * F.relu(margin - cos_sim) + labels * (1 - cos_sim))
    return loss


In [ ]:
from transformers import PreTrainedModel
from torch import nn

class ContrastiveModel(PreTrainedModel):
    def __init__(self, model):
        super().__init__(model.config)
        self.model = model

    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b, labels=None):
        # Получаем эмбеддинги для текстов A и B
        embeddings_a = self.model(input_ids=input_ids_a, attention_mask=attention_mask_a).last_hidden_state[:, 0, :]
        embeddings_b = self.model(input_ids=input_ids_b, attention_mask=attention_mask_b).last_hidden_state[:, 0, :]
        
        # Если есть метки, вычисляем потерю
        if labels is not None:
            loss = contrastive_loss(embeddings_a, embeddings_b, labels)
            return {"loss": loss, "embeddings_a": embeddings_a, "embeddings_b": embeddings_b}
        
        return {"embeddings_a": embeddings_a, "embeddings_b": embeddings_b}


In [ ]:
from transformers import Trainer, TrainingArguments

# Создаем кастомную модель
contrastive_model = ContrastiveModel(model)

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./contrastive_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

# Загрузка датасета
train_dataset = PairDataset("path/to/your/dataset.csv", tokenizer)

# Инициализация Trainer
trainer = Trainer(
    model=contrastive_model,
    args=training_args,
    train_dataset=train_dataset,
)

# Запуск обучения
trainer.train()


In [ ]:
from transformers import Trainer, TrainingArguments

# Создаем кастомную модель
contrastive_model = ContrastiveModel(model)

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./contrastive_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

# Загрузка датасета
train_dataset = PairDataset("path/to/your/dataset.csv", tokenizer)

# Инициализация Trainer
trainer = Trainer(
    model=contrastive_model,
    args=training_args,
    train_dataset=train_dataset,
)


# Запуск обучения
trainer.train()


In [ ]:
pip install transformers torch sentence-transformers pandas

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class TextSimilarityDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text_a = self.data.iloc[idx]["text_a"]
        text_b = self.data.iloc[idx]["text_b"]
        label = self.data.iloc[idx]["label"]

        # Токенизируем тексты
        encoding = self.tokenizer(
            text_a, text_b,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.float)
        }


In [ ]:
from transformers import AutoModel

# Загружаем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CosineSimilarityLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(CosineSimilarityLoss, self).__init__()
        self.margin = margin

    def forward(self, embeddings_a, embeddings_b, labels):
        cos_sim = F.cosine_similarity(embeddings_a, embeddings_b)
        # Вычисляем потери для похожих и непохожих пар
        loss = torch.mean((1 - labels) * torch.relu(self.margin - cos_sim) + labels * (1 - cos_sim))
        return loss

In [ ]:
from transformers import PreTrainedModel

class SimilarityModel(PreTrainedModel):
    def __init__(self, model):
        super().__init__(model.config)
        self.model = model
        self.loss_fn = CosineSimilarityLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        # Разделяем на две части
        embeddings = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        embeddings_a, embeddings_b = embeddings[::2], embeddings[1::2]
        
        if labels is not None:
            # Потеря для обучения
            loss = self.loss_fn(embeddings_a, embeddings_b, labels[::2])
            return {"loss": loss}
        
        return {"embeddings_a": embeddings_a, "embeddings_b": embeddings_b}


In [ ]:
from transformers import Trainer, TrainingArguments

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./similarity_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

# Загрузка датасета
train_dataset = TextSimilarityDataset("path/to/your/dataset.csv", tokenizer)

# Инициализация Trainer
trainer = Trainer(
    model=SimilarityModel(model),
    args=training_args,
    train_dataset=train_dataset,
)

# Запуск обучения
trainer.train()


In [ ]:
import pandas as pd
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

# Шаг 1: Загрузка данных
def load_data(file_path):
    data = pd.read_csv(file_path)
    # Проверим наличие необходимых колонок
    assert all(col in data.columns for col in ["text_a", "text_b", "label"]), "Необходимые колонки отсутствуют."
    return data

# Шаг 2: Очистка данных
def clean_data(data):
    # Удалим строки с пропущенными значениями
    data = data.dropna(subset=["text_a", "text_b", "label"])
    # Проверим, что метки имеют только допустимые значения (0 и 1)
    data = data[(data['label'] == 0) | (data['label'] == 1)]
    return data

# Шаг 3: Преобразование данных в формат InputExample
def create_examples(data):
    examples = [
        InputExample(texts=[row['text_a'], row['text_b']], label=float(row['label']))
        for _, row in data.iterrows()
    ]
    return examples

# Шаг 4: Создание DataLoader
def create_dataloader(examples, batch_size=16):
    dataloader = DataLoader(examples, shuffle=True, batch_size=batch_size)
    return dataloader

# Основной код для обработки и подготовки данных
file_path = "path/to/your/dataset.csv"
data = load_data(file_path)
data = clean_data(data)
examples = create_examples(data)
train_dataloader = create_dataloader(examples)

print(f"Количество примеров для обучения: {len(examples)}")
